In [64]:
import pandas as pd
import os
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
import json

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
nombre_experimento = 'LSTM_GRU_producto_optuna'
ventana_input = 12
ventana_output = 2




# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)



dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))
dataset_completo.head()

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000


In [65]:
ventas_producto_mes = dataset_completo.groupby(['Timestamp', 'product_id'])['tn'].sum()
ventas_producto_mes = ventas_producto_mes.reset_index()
ventas_producto_mes.set_index('Timestamp', inplace=True)
ventas_producto_mes

,product_id,tn
Timestamp,,
2017-01-01,20001,934.77222
2017-01-01,20002,550.15707
2017-01-01,20003,1063.45835
2017-01-01,20004,555.91614
2017-01-01,20005,494.27011
...,...,...
2019-12-01,21263,0.01270
2019-12-01,21265,0.05007
2019-12-01,21266,0.05121


In [ ]:
#Recuperar mejores hiperparametros de optimizacion
with open(os.path.join(os.path.join(carpeta_exp_base,nombre_carpeta_optuna), 'mejores_hiperparametros.json'), 'r') as f:
    best_params = json.load(f)
print(best_params)

units_lstm = [best_params['units_lstm_1'], best_params['units_lstm_2'], best_params['units_lstm_3']]
dense_units = best_params['dense_units']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']
epochs = best_params['epochs']

In [67]:
def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape) == 1:  # Si tenemos sólo una serie (univariado)
        array = array.reshape(-1, 1)
        cols = 1
    else:  # Multivariado
        fils, cols = array.shape

    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils - input_length - output_length + 1):
        X.append(array[i:i + input_length, :].reshape(input_length, cols))
        Y.append(array[i + input_length:i + input_length + output_length, -1].reshape(output_length, 1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [68]:
# Función para crear el modelo 
def crear_modelo(input_shape, units_lstm, dense_units, dropout_rate, learning_rate):
    model = Sequential()
    model.add(Bidirectional(LSTM(units_lstm[0], return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units_lstm[1], return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(GRU(units_lstm[2], return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(dense_units))
    model.add(Dense(ventana_output))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics=[MeanSquaredError()])

    return model


# Function to add additional features to the data
def add_features(data):
    df = pd.DataFrame(data, columns=['tn'])
    df.index = pd.to_datetime(df.index)
    df['lag_1'] = df['tn'].shift(1)
    df['lag_2'] = df['tn'].shift(2)
    df['ma_3'] = df['tn'].rolling(window=3).mean()
    df['ma_6'] = df['tn'].rolling(window=6).mean()
    df['std_3'] = df['tn'].rolling(window=3).std()
    df['trend'] = range(len(df))
    df['month'] = df.index.month
    df['sin_month'] = np.sin(2 * np.pi * df['month']/12)
    df['cos_month'] = np.cos(2 * np.pi * df['month']/12)
    df.dropna(inplace=True)
    return df.values

In [69]:
# Función para la optimización con Optuna
def objective(trial):
    # Definir hiperparámetros a optimizar
    units_lstm = [
        trial.suggest_int('units_lstm_1', 32, 128),
        trial.suggest_int('units_lstm_2', 32, 128),
        trial.suggest_int('units_lstm_3', 32, 128)
    ]
    
    dense_units = trial.suggest_int('dense_units', 8, 32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)


    lista_productos_LSTM = []
    lista_predicciones_LSTM = []
    loss_list = []

    for producto in lista_productos_mayores_ventas:
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
        ventas_mes_por_producto.drop(columns=['product_id'], inplace=True)
        
        ventas_mes_por_producto_features = add_features(ventas_mes_por_producto)
        
        if len(ventas_mes_por_producto_features) >= (ventana_input + ventana_output):
            # Escalar valor
            scaler = RobustScaler()
            ventas_mes_por_producto_features = scaler.fit_transform(ventas_mes_por_producto_features)
            
            # Formatear valores para input LSTM
            X, Y = crear_dataset_supervisado(ventas_mes_por_producto_features, ventana_input, ventana_output)

            # Separar datos en entrenamiento y test
            train_size = len(X) - ventana_test
            X_train, X_test = X[:train_size], X[train_size:]
            Y_train, Y_test = Y[:train_size], Y[train_size:]
            
            early_stop = EarlyStopping(monitor='loss', patience=20)
            reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-6)

            # Crear y ajustar el modelo LSTM
            model = crear_modelo((ventana_input, X.shape[2]),units_lstm,dense_units, dropout_rate, learning_rate)
            model.fit(X_train, Y_train, epochs=150, batch_size=24, callbacks=[early_stop, reduce_lr], verbose=0)

            # Evaluar el modelo
            loss = model.evaluate(X_test, Y_test, verbose=0)
            loss_list.append(loss)

    return np.mean(loss_list)

In [70]:
# Ejecución de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_trials)

# Guardar los mejores hiperparámetros
best_params = study.best_params
with open(os.path.join(carpeta_exp, 'mejores_hiperparametros.json'), 'w') as f:
    json.dump(best_params, f)

print("Mejores hiperparámetros encontrados: ", best_params)

[I 2024-07-13 16:09:05,027] A new study created in memory with name: no-name-dc65b163-e092-4a5d-96c6-1d5689efc1db
[I 2024-07-13 16:12:56,795] Trial 0 finished with value: 0.053441877830428655 and parameters: {'units_lstm_1': 42, 'units_lstm_2': 96, 'units_lstm_3': 69, 'dense_units': 18, 'dropout_rate': 0.3637276346877524, 'learning_rate': 0.004313455865153597}. Best is trial 0 with value: 0.053441877830428655.
[I 2024-07-13 16:16:45,453] Trial 1 finished with value: 0.02945935312783587 and parameters: {'units_lstm_1': 38, 'units_lstm_2': 70, 'units_lstm_3': 88, 'dense_units': 20, 'dropout_rate': 0.33400850548535105, 'learning_rate': 0.007113846383897537}. Best is trial 1 with value: 0.02945935312783587.
[I 2024-07-13 16:20:44,716] Trial 2 finished with value: 0.03227352982018298 and parameters: {'units_lstm_1': 56, 'units_lstm_2': 68, 'units_lstm_3': 113, 'dense_units': 31, 'dropout_rate': 0.26107897506285727, 'learning_rate': 0.007835039032041942}. Best is trial 1 with value: 0.029459

Mejores hiperparámetros encontrados:  {'units_lstm_1': 110, 'units_lstm_2': 112, 'units_lstm_3': 45, 'dense_units': 25, 'dropout_rate': 0.13752145556267067, 'learning_rate': 0.006048498330212748}
